In [6]:
from PIL import Image
import os
import json
from shapely.geometry import box, mapping
from geojson import Feature
import rasterio

# Deaktiviert die Begrenzung für große Bilder
Image.MAX_IMAGE_PIXELS = None

def split_tif_to_tiles(input_tif, output_folder, tile_size=(320, 320)):
    """
    Zerlegt eine TIFF-Datei in kleinere Kacheln und erstellt nur die GeoJSON-Datei mit den Geoinformationen der Kacheln.

    :param input_tif: Pfad zur Eingabe-TIFF-Datei
    :param output_folder: Ordner, in dem die GeoJSON-Datei gespeichert wird
    :param tile_size: Tuple mit der Breite und Höhe der Kacheln (Standard: 320x320)
    """
    # Sicherstellen, dass der Ausgabeordner existiert
    os.makedirs(output_folder, exist_ok=True)

    # Öffne die TIFF-Datei einmalig mit Rasterio, um Metadaten zu extrahieren
    with rasterio.open(input_tif) as src:
        width, height = src.width, src.height
        bounds = src.bounds
        tile_width, tile_height = tile_size

        # Berechne die Pixelgröße in den geographischen Koordinaten
        pixel_size_x = (bounds.right - bounds.left) / width
        pixel_size_y = (bounds.top - bounds.bottom) / height

        # GeoJSON-Datei öffnen und Features direkt schreiben
        geojson_path = os.path.join(output_folder, 'tiles_metadata_from_geotiff.geojson')
        with open(geojson_path, 'w') as geojson_file:
            geojson_file.write('{"type": "FeatureCollection", "features": [')

            tile_count = 0
            with Image.open(input_tif) as img:
                # Schleife über die Kacheln
                for top in range(0, height, tile_height):
                    for left in range(0, width, tile_width):
                        # Berechne die Kachelgrenzen
                        right = min(left + tile_width, width)
                        bottom = min(top + tile_height, height)

                        # Schneide die Kachel
                        tile = img.crop((left, top, right, bottom))

                        # (Optional: Speichere die Kachel, wenn notwendig)
                        tile_filename = f"tile_{tile_count:04d}.png"
                        tile.save(os.path.join(output_folder, tile_filename))

                        # Berechne die geographischen Koordinaten der Kachel
                        left_geo = bounds.left + left * pixel_size_x
                        top_geo = bounds.top - top * pixel_size_y
                        right_geo = left_geo + (right - left) * pixel_size_x
                        bottom_geo = top_geo - (bottom - top) * pixel_size_y

                        # Erstelle das GeoJSON-Feature
                        geometry = box(left_geo, bottom_geo, right_geo, top_geo)
                        feature = Feature(geometry=mapping(geometry), properties={
                            "tile_id": f"tile_{tile_count:04d}"  # Eindeutige ID für die Kachel
                        })

                        # Schreibe das Feature direkt in die Datei
                        if tile_count > 0:
                            geojson_file.write(',')  # Komma zwischen Features
                        geojson_file.write(json.dumps(feature))

                        tile_count += 1

            # Schließe die GeoJSON-Struktur
            geojson_file.write(']}')

        print(f"{tile_count} Kacheln analysiert, als PNG gespeichert und GeoJSON-Datei mit den Geoinformationen gespeichert unter '{output_folder}'")

# Beispielverwendung
input_tif = r"C:\Users\bilge\OneDrive\Masaüstü\ML Wärmelyse\DOP10_Mosaik.tif"  # Pfad zur Eingabe-TIFF-Datei
output_folder = r"C:\Users\bilge\OneDrive\Masaüstü\waermelyse\machine_learning\Georeferencing_GeoTiff\tif_output_trees"  # Ordner für die GeoJSON-Datei
split_tif_to_tiles(input_tif, output_folder)



ModuleNotFoundError: No module named 'rasterio'